In [47]:
from azureml.core import Workspace, Experiment, Run
from sklearn.externals import joblib
from azureml.core.authentication import ServicePrincipalAuthentication


spa=ServicePrincipalAuthentication("9d396536-b01d-424e-ace8-2a339bc8e502", "e9311799-43b1-4a27-9e08-266cc96a2c05","K_?XRu6bSX/DV3.WYsuS3cwxjgOg4DzO" , _enable_caching=False)
ws = Workspace.get(name='acc_ws',auth=spa, subscription_id='f30951ea-6926-40a7-9c19-adeba1c67ec4', resource_group='acc_ws_rg')



 ### Downloading and Uploading Data From Blob Storage

In [49]:
from azureml.core import Datastore

def_data_store = ws.get_default_datastore()

In [50]:
# Creating a new datastore
Datastore.register_azure_blob_container(workspace=ws, 
                                                      datastore_name='my_datastore', 
                                                      container_name='azureml-blobstore-38073728-8ae8-4d27-be99-de5495d1946f',
                                                      account_name='accws5803293148', 
                                                      account_key='UkZb7f469TziWP0jA50wAn6G4+Q+uHlKZtZ78Nn6Eq7P/BI6NRiivZkVoNv7YJYgRjeW0gYQ1om0iXtWzns3UQ==',
                                                      create_if_not_exists=True)

In [23]:
# Changing default datastore
ws.set_default_datastore('my_datastore')

In [42]:
mydir="C:/Users/ygulf/Desktop/Intertech/data"

In [24]:
def_data_store.upload(src_dir=mydir,
          target_path='batchscoring/data',
          overwrite=True,
          show_progress=True)

Uploading an estimated of 1 files
Uploading C:/Users/ygulf/Desktop/Intertech/data\bank-additional-full.csv
Uploaded C:/Users/ygulf/Desktop/Intertech/data\bank-additional-full.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_46af689c254d47a58e3e9ffc3c2ea4c0

In [25]:
def_data_store.download(target_path="abc",
           show_progress=True,prefix="batchscoring/models/propensity_model.pkl")

Downloaded batchscoring/models/propensity_model.pkl, 1 files out of an estimated total of 1


1

### Creating a Pipeline

In [29]:
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import PipelineData


data_dir = DataReference(datastore=def_data_store,
                             data_reference_name="input_data",
                         path_on_datastore="batchscoring/data/",
                             mode="download")

model_dir = DataReference(datastore=def_data_store,
                          data_reference_name="input_model",
                          path_on_datastore="batchscoring/models/propensity_model.pkl",
                          mode="download")



pipeline_dir = PipelineData(name="pipelineoutput",
                          datastore=def_data_store)

output_dir = PipelineData(name="auc",
                          datastore=def_data_store,pipeline_output_name
="auc")

output_rows = PipelineData(name="rows",
                          datastore=def_data_store,pipeline_output_name
="rows")



In [51]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpuclusterpipe"

# Verify that cluster does not exist already
try:
    comp = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    comp = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

comp.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


In [52]:
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

cd = CondaDependencies.create(conda_packages=['scikit-learn','pandas','numpy'], pip_packages=['azureml-sdk','azureml-core','azure'])

# Runconfig
amlcompute_run_config = RunConfiguration(conda_dependencies=cd)
amlcompute_run_config.environment.docker.enabled = True
amlcompute_run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

In [36]:
from azureml.pipeline.core.graph import PipelineParameter

param = PipelineParameter(name="threshold",default_value=70)


In [37]:
from azureml.pipeline.steps import PythonScriptStep

batch_score_step = PythonScriptStep(
    name="batch_scoring",
    script_name="batch_scoring.py",
    arguments=["--dataset_path", data_dir,
               "--model_name", model_dir,
               "--pipeline_dir", pipeline_dir],
    compute_target=comp,
    inputs=[data_dir,model_dir],
    outputs=[pipeline_dir],
    runconfig=amlcompute_run_config,
source_directory='C:/Users/ygulf/Desktop/Intertech')

In [38]:
get_rows = PythonScriptStep(
    name="get_rows",
    script_name="get_rows.py",
    arguments=["--pipeline_dir", pipeline_dir,
               "--threshold",param,"--output_rows",output_rows],
    compute_target=comp,
    inputs=[pipeline_dir,data_dir],
    outputs=[output_rows],
    runconfig=amlcompute_run_config,
    source_directory="C:/Users/ygulf/Desktop/Intertech")

In [43]:
logging = PythonScriptStep(
    name="logging",
    script_name="log.py",
    arguments=["--pipeline_dir", pipeline_dir,
              "--output_dir",output_dir],
    compute_target=comp,
    inputs=[pipeline_dir],
    outputs=[output_dir],
    runconfig=amlcompute_run_config,
    source_directory="C:/Users/ygulf/Desktop/Intertech")

In [44]:
from azureml.pipeline.core import Pipeline

# Run the pipeline
pipeline = Pipeline(workspace=ws, steps=[batch_score_step,get_rows,logging])
pipeline_run = Experiment(ws, 'batch_scoring').submit(pipeline,pipeline_params={"threshold" : 0})

# Wait for the run to finish (this might take several minutes)
pipeline_run.wait_for_completion(show_output=True)

# Download and review the output

pipeline_run.get_pipeline_output('rows').download("./rows")

pipeline_run.get_pipeline_output('auc').download("./auc")
#pipeline_run.find_step_run('logging')[0].get_output_data('auc').download("./model_auc")

#pipeline_run.find_step_run('get_rows')[0].get_output_data('rows').download("./model_auc")

WARNING - The 'pipeline_params' argument is deprecated. Please use 'pipeline_parameters' instead.


Created step batch_scoring [77bf6419][594681df-9d2d-41a5-9958-4d2512048627], (This step will run and generate new outputs)Created step get_rows [b17fed92][556e3cf6-bccf-428c-bc72-91398982d785], (This step will run and generate new outputs)
Created step logging [8e44f124][6f13c2ab-2de7-4db6-b64b-499fc9461929], (This step will run and generate new outputs)

Using data reference input_data for StepId [32ffea91][2e767a69-ea7f-41df-a8c3-7a9581bf3b44], (Consumers of this data are eligible to reuse prior runs.)Using data reference input_model for StepId [a3763baf][3c893664-0df8-402f-83bf-13e45a3374fa], (Consumers of this data are eligible to reuse prior runs.)
Using data reference input_data for StepId [fdd3f9a3][2e767a69-ea7f-41df-a8c3-7a9581bf3b44], (Consumers of this data are eligible to reuse prior runs.)

Submitted PipelineRun 0fc29002-df71-4a75-8563-201d628b7696
Link to Azure Machine Learning studio: https://ml.azure.com/experiments/batch_scoring/runs/0fc29002-df71-4a75-8563-201d628b769



The experiment completed successfully. Finalizing run...
Cleaning up all outstanding Run operations, waiting 300.0 seconds
1 items cleaning up...
Cleanup took 0.0008821487426757812 seconds
Starting the daemon thread to refresh tokens in background for process with pid = 211

Streaming azureml-logs/75_job_post-tvmps_9504c54b5b63c36bc501aa8d8685d355272b667bcbc581239140ec7d299ee042_d.txt
bash: /azureml-envs/azureml_200871206332612ac643058f820b27a3/lib/libtinfo.so.5: no version information available (required by bash)
Starting job release. Current time:2019-12-07T19:31:35.885605
Logging experiment finalizing status in history service.
Starting the daemon thread to refresh tokens in background for process with pid = 241
Job release is complete. Current time:2019-12-07T19:31:38.758036

StepRun(batch_scoring) Execution Summary
StepRun( batch_scoring ) Status: Finished
{'runId': '576368d1-131c-4587-a7d9-0dfccf9e1ddd', 'target': 'cpuclusterpipe', 'status': 'Completed', 'startTimeUtc': '2019-1

StepRun( get_rows ) Status: Running

Streaming azureml-logs/55_azureml-execution-tvmps_9504c54b5b63c36bc501aa8d8685d355272b667bcbc581239140ec7d299ee042_d.txt
2019-12-07T19:33:33Z Starting output-watcher...
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_b32ac052eeaaa849bd7bd6b984e87c12
Digest: sha256:63daf73610e7d1366d3613380f12d79a90bb08348d3692210a2018645a7a84be
Status: Image is up to date for accws2463cd3b.azurecr.io/azureml/azureml_b32ac052eeaaa849bd7bd6b984e87c12:latest
57c64da823f2077a0366bf9c5e89626183d343edb3dc20248d904b933969b846
2019/12/07 19:33:36 Version: 3.0.01059.0002 Branch: master Commit: e8f402a4
2019/12/07 19:33:37 sshd runtime has already been installed in the container
ssh-keygen: /azureml-envs/azureml_200871206332612ac643058f820b27a3/lib/libcrypto.so.1.0.0: no version information available (required by ssh-keygen)
ssh-keygen: /azureml-envs/azureml_200871206332612ac643058f820b27a3/lib/libcrypto.so.1.0.0: no version information availabl




StepRunId: dda889dd-4871-4fb1-a29c-e479bab88e8f
Link to Portal: https://ml.azure.com/experiments/batch_scoring/runs/dda889dd-4871-4fb1-a29c-e479bab88e8f?wsid=/subscriptions/f30951ea-6926-40a7-9c19-adeba1c67ec4/resourcegroups/acc_ws_rg/workspaces/acc_ws

StepRun(logging) Execution Summary
StepRun( logging ) Status: Finished
{'runId': 'dda889dd-4871-4fb1-a29c-e479bab88e8f', 'target': 'cpuclusterpipe', 'status': 'Completed', 'startTimeUtc': '2019-12-07T19:32:30.762724Z', 'endTimeUtc': '2019-12-07T19:33:31.011909Z', 'properties': {'azureml.runsource': 'azureml.StepRun', 'ContentSnapshotId': 'eb9647af-7cdf-4a48-a3db-9e8fe79e1091', 'StepType': 'PythonScriptStep', 'ComputeTargetType': 'AmlCompute', 'azureml.pipelinerunid': '0fc29002-df71-4a75-8563-201d628b7696', '_azureml.ComputeTargetType': 'amlcompute', 'AzureML.DerivedImageName': 'azureml/azureml_b32ac052eeaaa849bd7bd6b984e87c12', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json

1

In [ ]:
published_pipeline = pipeline.publish(name="testing_pipeline", continue_on_step_failure=True)


In [43]:
from azureml.core.authentication import InteractiveLoginAuthentication
import requests

auth = InteractiveLoginAuthentication()
aad_token = auth.get_authentication_header()

rest_endpoint1 = published_pipeline.endpoint

print("You can perform HTTP POST on URL {} to trigger this pipeline".format(rest_endpoint1))

# specify the param when running the pipeline
response = requests.post(rest_endpoint1, 
                         headers=aad_token, 
                         json={"ExperimentName": "My_Pipeline1",
                               "RunSource": "SDK",
                               "ParameterAssignments": {"pipeline_arg": 45}})
run_id = response.json()["Id"]

print(run_id)

You can perform HTTP POST on URL https://westeurope.aether.ms/api/v1.0/subscriptions/f30951ea-6926-40a7-9c19-adeba1c67ec4/resourceGroups/acc_ws_rg/providers/Microsoft.MachineLearningServices/workspaces/acc_ws/PipelineRuns/PipelineSubmit/7976cc4b-8834-4f20-a081-7d5af31d276a to trigger this pipeline
dea0647d-25d0-4aad-b2cb-f3e00d474a26
